In [1]:
# Install searchtweets wrapper for the premium API

!pip install searchtweets

In [11]:
# Import libraries

from searchtweets import load_credentials, gen_rule_payload, ResultStream
import pandas as pd
import json
import yaml

In [168]:
premium_search_args = load_credentials("/Users/dougl/.secret/twitter_keys_30days.yaml",
                                       yaml_key="search_tweets_api",
                                       env_overwrite=False)
print(premium_search_args)

Grabbing bearer token from OAUTH


{'bearer_token': 'AAAAAAAAAAAAAAAAAAAAAGCjCgEAAAAA3u1GCjoHUSbgcaYY1SP2MG9WIbo%3DwfzWYcrDUyqs0msK9VGiQCny7qBQq5jn4sWfolTSqmUj4eBVbU', 'endpoint': 'https://api.twitter.com/1.1/tweets/search/30day/flatironcapstone.json', 'extra_headers_dict': None}


In [199]:
# Set up rule to obtain tweets for last month, excluding time period  already covered by standard twitter API

rule = gen_rule_payload("(#americanairlines OR #americanair OR @AmericanAir) -is:retweet -is:reply -RT lang:en",
                       results_per_call=500,
                       from_date="2020-01-25 03:00",
                        to_date="2020-02-25")

In [200]:
rs = ResultStream(rule_payload=rule,
                  max_results=100000,
                  **premium_search_args)
print(rs)

ResultStream: 
	{
    "username": null,
    "endpoint": "https://api.twitter.com/1.1/tweets/search/30day/flatironcapstone.json",
    "rule_payload": {
        "query": "(#americanairlines OR #americanair OR @AmericanAir) -is:retweet -is:reply -RT lang:en",
        "maxResults": 500,
        "toDate": "202002250000",
        "fromDate": "202001250300"
    },
    "tweetify": true,
    "max_results": 100000
}


In [201]:
rs.stream()

<generator object ResultStream.stream at 0x000001CBB5323888>

In [202]:
iD = []
datetime = []
text = []
retweets = []

for tweet in rs.stream():
    iD.append(tweet['id'])
    datetime.append(tweet['created_at'])
    try:
        text.append(tweet['extended_tweet']['full_text'])
    except:
        try:
            text.append(tweet['full_text'])
        except:
            text.append(tweet['text'])
    retweets.append(tweet['retweet_count'])

In [203]:
df = pd.DataFrame([])
df = pd.concat([df, pd.Series(iD), pd.Series(datetime), pd.Series(text), pd.Series(retweets)], axis=1)
df.columns = ['id', 'datetime', 'text', 'retweets']
df

,id,datetime,text,retweets
0,1232091470154338304,Mon Feb 24 23:54:32 +0000 2020,Joining us now Beverly Bass who was the first ...,1
1,1232089893628776448,Mon Feb 24 23:48:16 +0000 2020,More @AmericanAir mechanical delays. Yo @South...,0
2,1232087057352146944,Mon Feb 24 23:37:00 +0000 2020,Check out #KidsCubsCactus for your daily dose ...,3
3,1232086687125311490,Mon Feb 24 23:35:31 +0000 2020,2 days in a row. Travel nightmares. @American...,0
4,1232086061695651840,Mon Feb 24 23:33:02 +0000 2020,Hey @AmericanAir. Making these seats smaller s...,0
...,...,...,...,...
11085,1220918175329673216,Sat Jan 25 03:55:51 +0000 2020,#AmericanAirlines currently 4.5 hours delayed ...,1
11086,1220911582827491335,Sat Jan 25 03:29:39 +0000 2020,Ouch @AmericanAir flight 380 from LGA! Very ab...,0
11087,1220907874651688962,Sat Jan 25 03:14:55 +0000 2020,"...48°F in #WashingtonDC. Oh boy, that's too h...",0
11088,1220907640248832001,Sat Jan 25 03:13:59 +0000 2020,"Just flew @AmericanAir and it was the hottest,...",0


In [204]:
# Convert final_df to a .csv file, and save in current directory

df.to_csv('twitter_30_days.csv', index=False)